In [ ]:
import numpy as np
from datetime import date
from PIL import Image, ImageDraw, ImageFont

### Блок расчетов

In [2]:
def birthday():
    """Ввод даты рождения и имени

    Returns:
        the tuple: (дата рождения в формате date, имя)
    """
    date_components = input('Enter a date formatted as ДД.MM.ГГГГ: ').split('.')

    day, month, year = [int(item) for item in date_components]
    birth_day = date(year, month, day)

    name = input('Введите имя')

    return (birth_day, name)

def num_to_single(num):
    """Рассчитывает одно число из суммы цифр числа num
    Args:
        num (int): number
    Returns:
        int: one number
    """
    string = str(num)

    while len(string)>1:
        string = str(sum([int(item) for item in string]))
        
    return int(string)


def get_main(data):
    """Возвращает основные числа, рассчитанные по дате рождения
    Args:
        data: (дата рождения) в формате date  
    Returns:
        tuple: (day, month, year, sum_3, sum_4)  в приведенном к одному числу формате
    """
    a = [num_to_single(item) for item in (data.day, data.month, data.year)]
    
    for i in range(2):
        a.append(num_to_single(sum(a)))

    return tuple(a)


def get_additional(data):
    """Возвращает дополнительные числа, рассчитанными попарно из основных
    Args:
        data (tuple): main_data
    Returns:
        tuple: (day_month, month_year, year_sum3, sum3_sum4, sum4_day)
    """

    a=[num_to_single(data[i]+data[i+1]) for i in range(len(data)-1)]
    a.append(num_to_single(data[0]+data[-1]))
    return tuple(a)


### Parameters block

In [3]:
# Шаблоны и результаты
template_path = '/Users/user/Desktop/Python/Jupiter_Notebook/Numerolog/Page_templates/'
result_path = '/Users/user/Desktop/Python/Jupiter_Notebook/Numerolog/Results/'

In [27]:
# Шрифты для заполнения форм 

# Характеристики для заголовков
header_font = {
    "family": 'Gilroy-Semibold.ttf',
    "size": 30,
    "color": '#E3DDD0',
    "stroke_fill": None,
    "stroke_width": None
}

# Характеристики для футера
footer_font = {
    "family": 'Gilroy-Semibold.ttf',
    "size": 45,
    "color": '#E3DDD0',
    "stroke_fill": None,
    "stroke_width": None
}

# Характеристики для основного текста
body_font = {
    "family": "Calibri",
    "size": 14,
    "color": "black",
    "stroke_fill": None,
    "stroke_width": None
}

# Характеристики для табличного текста (для звезд, таблиц и пр.)
main_font = {
    "family": 'Gilroy-Bold.ttf',
    "size": 35,
    "color": "#E6DFD2",
    "stroke_fill": None,
    "stroke_width": None
}
additional_font = {
    "family": 'Gilroy-Medium.ttf',
    "size": 25,
    "color": "#E6DFD2",
    "stroke_fill": 'black',
    "stroke_width": 1
}
mission_font = {
    "family": 'Gilroy-Semibold.ttf',
    "size": 40,
    "color": "#FF693A",
    "stroke_fill": None,
    "stroke_width": None
}
money_main_font = {
    "family": 'Gilroy-Semibold.ttf',
    "size": 40,
    "color": "#04070F",
    "stroke_fill": None,
    "stroke_width": None
}
pifagor_font = header_font
money_font = main_font

In [5]:
# Параметры и координаты "посадочных" квадратов для заполнения звезды
main_width, main_height = 55, 55
additional_width, additional_height = 45, 45
mission_width, mission_height = 60, 60

main_coord = np.array([[81, 268, 458, 388, 145], [290, 148, 290, 518, 518]])
additional_coord = np.array([[221, 323, 350, 273, 200], [299, 299, 389, 444, 389]])
mission_coord = np.array([[268], [351]])

In [6]:
# Параметры и координаты "посадочных" фигкр для заполнения header and footer
header_width, header_height = 506, 78
header_coord = np.array([[45], [35]])

footer_width, footer_height = 26, 38
footer_coord = np.array([[183, 234, 285, 336, 387], [706, 706, 706, 706, 706]])

### Блок расчета координат

In [7]:
def get_pos(data, dimension, img=None, my_font=None, coordinates=None):
    """Возвращает координаты для ввода данных в объект img
    Args:
        data (tuple): данные для вставки в изображение
        dimension (tuple): (ширина,  высота) изображения
        img (text): ссылка на файл изображения
        my_font (dict): параметры шрифта
        coordinates (np.array): координаты мест вставки на шаблоне страницы

    Returns:
        a (array): координаты для вставки данных data в шаблон страницы
    """

    # Создаем изображение
    im = Image.open(img) if img else Image.new('RGB', dimension, color='white')
    
    width, height = im.size
    draw = ImageDraw.Draw(im)
    font = ImageFont.truetype(font=my_font['family'], size=my_font['size'])
    X = []
    Y = []

    for item in data:
        # Получаем размеры текста с помощью textbbox
        text_bbox = draw.textbbox((0, 0), str(item), font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        # Вычисляем координаты для выравнивания текста по центру
        x = (width - text_width) / 2
        y = (height - text_height) / 2
        X.append(x)
        Y.append(y)
        a = np.array([X, Y])
    
    im.close()
    b = coordinates + a
    return [(float(b[0][i]), float(b[1][i])) for i in range(len(b[0]))]

### Блок вывода на экран и записи в файл

In [8]:
def text2star(im, data, coords, my_font=None):
    """Вывод на экран и добавление данных к изображению
    Args:
        im : открытый объект класса Image (изображение)
        data (list): данные для добавления в изображение
        coords (tuple): координаты для добавления данных в изображение
        my_font (dict): параметры шрифта
    """

    draw_text = ImageDraw.Draw(im)
    font = ImageFont.truetype(font=my_font['family'], size=my_font['size'])

    for item in zip(data, coords):
        text = item[0]
        place = item[1]
        draw_text.text(
            place,
            str(text),
            font=font,
            fill=(my_font['color']),
            stroke_fill = my_font['stroke_fill'],
            stroke_width = my_font['stroke_width']
            )
    return im

In [9]:
# Ввод даты рождения и имени
birth_day, name  = birthday()

In [10]:
# Формирование переменных для добавления на 1 страницу ("звезда")
header_data = (f'{name} {birth_day.strftime("%d.%m.%Y")}',)
main_data = get_main(birth_day)
additional_data = get_additional(main_data)
mission_data = (num_to_single(sum(main_data)),)
footer_data = main_data

In [11]:
# Расчет координат для данных в шаблоне
header_coordinates = get_pos(header_data, (header_width, header_height), img=None, my_font=header_font, coordinates=header_coord)
main_coordinates = get_pos(main_data, (main_width, main_height), img=None, my_font=main_font, coordinates=main_coord)
additional_coordinates = get_pos(additional_data, (additional_width, additional_height), img=None, my_font=additional_font, coordinates=additional_coord)
mission_coordinates = get_pos(mission_data, (mission_width, mission_height), img=None, my_font=mission_font, coordinates=mission_coord)
footer_coordinates = get_pos(footer_data, (footer_width, footer_height), img=None, my_font=footer_font, coordinates=footer_coord)

In [12]:
img = template_path + 'A4 - 1.jpg'
with Image.open(img) as im:
    text2star(im, header_data, header_coordinates, my_font=header_font)
    text2star(im, main_data, main_coordinates, my_font=main_font)
    text2star(im, additional_data, additional_coordinates, my_font=additional_font)
    text2star(im, mission_data, mission_coordinates, my_font=mission_font)
    first_page = text2star(im, footer_data, footer_coordinates, my_font=footer_font)

    first_page.show()
    first_page.save(f'{result_path}{name}_page_01.jpg')

### Создание страницы "Таблица Пифагора"

In [13]:
def get_pif_additional(data):
    """Рассчитывает и возвращает кортеж из дня рождения и дополнительных чисел 

    Args:
        data (date): дата рождения в формате date

    Returns:
        tuple: (pif_add_1, pif_add_2, pif_add_4, pif_add_4, pif_add_5)
        четыре или пять дополнительных чисел в зависимости от года рождения
    """
    pif1 = sum([int(item) for item in data.strftime("%d%m%Y")])
    pif2 = num_to_single(pif1) if pif1 not in(11, 22, 33, 44) else pif1
    pif3 = pif1 - 2 * int(str(data.day)[0]) if data.year < 2000 else 19
    print(pif3)
    pif4 = num_to_single(pif3) if data.year < 2000 else pif1 + pif3
    pif5 = 0 if data.year < 2000 else num_to_single(pif4)

    result =((int(data.strftime("%d%m%Y")), pif1, pif2, pif3, pif4) if data.year < 2000 
                else (int(data.strftime("%d%m%Y")), pif1, pif2, pif3, pif4, pif5))
    
    return result


def get_pif_dict(data):
    """Принимает дату рождения и дополнительные числа Пифагора
        возвращает словарь с количеством цифр в каждом квадрате

    Args:
        data tuple: кортеж из даты и доп. чисел

    Returns:
        dict: количество цифр от 1 до 9 в каждом квадрате
    """
    my_data = "".join([str(item) for item in data])
    pif_dict = {i-1:str(i) * my_data.count(str(i)) for i in range(1, 10)}
    pif_data = tuple(val if val else '0' for val in pif_dict.values())

    return pif_data

    
def matrix_from_grid(initial_coord, step_coord):
    """Подсчетывает координаты ячеек сетки по начальным значениям и шагу

    Args:
        initial_coord (tuple): начальные координаты сетки ячеек для текста
        step_coord (tuple): смещения по x y координат ячеек

    Returns:
        array: массив координат ячеек
    """
    x, y = initial_coord
    delta_x, delta_y = step_coord
    a = np.array([[x]*9, [y]*9])
    row_addition = np.array(list(np.arange(3) * delta_y)*3)

    for j in range(3,6):
        a[0][j] += delta_x

    for j in range(6,9):
        a[0][j] += 2 * delta_x

    a[1] += row_addition
    return a 

In [14]:
# Формирование данных для вставки в шаблон Таблица Пифагора
a = get_pif_additional(birth_day)
header_data = (" ".join((birth_day.strftime("%d.%m.%Y"), *tuple(map(str, a[1:])))),)
pifagor_data = get_pif_dict(a)

15


In [15]:
pifagor_width, pifagor_height = 120, 78
pifagor_x, pifagor_y = 67, 214
pifagor_stepx, pifagor_stepy = 170, 172

pifagor_coord = matrix_from_grid((pifagor_x, pifagor_y), (pifagor_stepx, pifagor_stepy))

In [16]:
# Формирование координат для вставки в шаблон Таблица Пифагора
header_coordinates = get_pos(header_data, (header_width, header_height), img=None, my_font=header_font, coordinates=header_coord)
pifagor_coordinates = get_pos(pifagor_data, (pifagor_width, pifagor_height), img=None, my_font=pifagor_font, coordinates=pifagor_coord)

In [17]:
img = template_path + 'A4 - 2.jpg'
with Image.open(img) as im:
    text2star(im, pifagor_data, pifagor_coordinates, my_font=pifagor_font)
    pifogor_page = text2star(im, header_data, header_coordinates, my_font=header_font)

    pifogor_page.show()
    pifogor_page.save(f'{result_path}{name}_page_02.jpg')

### Создание страницы "Денежный треугольник"

In [18]:
# Формирование данных для вставки в шаблон Денежный треугольник
my_data = (additional_data[1], main_data[2], additional_data[2])
temp = get_additional(my_data)

money_main_data = (additional_data[1],)
money_data = (*temp[:2], my_data[2], temp[2], my_data[1])

In [33]:
money_main_width, money_main_height = 60, 60
money_width, money_height = 55, 55

money_main_coord = np.array([[265], [304]])
money_coord = np.array([[145, 392, 462, 270, 75], [551, 551, 687, 687, 687]])

In [34]:
# Формирование координат для вставки в шаблон Таблица Пифагора
money_main_coordinates = get_pos(money_main_data, (money_main_width, money_main_height), img=None, my_font=money_main_font, coordinates=money_main_coord)
money_coordinates = get_pos(money_data, (money_width, money_height), img=None, my_font=money_font, coordinates=money_coord)

In [ ]:
img = template_path + 'A4 - 3.jpg'

with Image.open(img) as im:
    text2star(im, money_main_data, money_main_coordinates, my_font=money_main_font)
    money_page = text2star(im, money_data, money_coordinates, my_font=money_font)

    money_page.show()
    money_page.save(f'{result_path}{name}_page_03.jpg')

In [41]:
img = template_path + 'A4 - 3.png'
img = "/Users/user/Desktop/Python/Jupiter_Notebook/Numerolog/Results/Евгений_page_03.jpg"
with Image.open(img) as image:
    dpi = image.info.get("dpi", (72, 72))  # (x-dpi, y-dpi)
    print(f"DPI изображения: {dpi[0]}x{dpi[1]}")

DPI изображения: 72x72
